# Imports

In [18]:
import numpy as np
from glob import glob
import torch
import os, sys, time, csv
import re
import random
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as ipyw
import inspect
import pickle
%matplotlib inline

# Process image stacks into feature vector matrices using pre-trained model (for LSTM input).

In [19]:
# Torch imports.
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torchvision.models.resnet

# Visualization imports
from IPython import display


In [20]:

# Part A: Load in model pre-trained on individual images. Note that pre-training may not even be necessary, as I was only
# changing the weights on the last (new) fully connected layer, and that layer will likely be gotten rid of. 
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
del model.classifier

# Assemble image pathnames and understand class breakdown in dataset

In [33]:
# Note down train, validation and test directories.
topDir = 'Split_Data'

train_dir = topDir + "/train"
valid_dir = topDir + "/valid"
test_dir = topDir + "/test"

# Check quantities of train, validation and test images
train_images = np.array(glob(train_dir + "/*/*"))
valid_images = np.array(glob(valid_dir + "/*/*"))
test_images = np.array(glob(test_dir + "/*/*"))

# Check relative percentages of image types
train_images_absent = np.array(glob(train_dir + "/Absent/*"))
train_images_present = np.array(glob(train_dir + "/Present/*"))

valid_images_absent = np.array(glob(valid_dir + "/Absent/*"))
valid_images_present = np.array(glob(valid_dir + "/Present/*"))

test_images_absent = np.array(glob(test_dir + "/Absent/*"))
test_images_present = np.array(glob(test_dir + "/Present/*"))

num_train_images = len(train_images)
num_valid_images = len(valid_images)
num_test_images = len(test_images)

print("There are {} training stacks, {} validation stacks and {} test stacks.".format(len(train_images),len(valid_images),len(test_images)))
print("For the training stacks, {pos:=.1f}% ({pos2}) are positive and {neg:=.1f}% ({neg2}) are negative.".format(pos=len(train_images_present)/len(train_images)*100, pos2=len(train_images_present),neg=len(train_images_absent)/len(train_images)*100, neg2=len(train_images_absent)))
print("For the validation stacks, {pos:=.1f}% ({pos2}) are positive and {neg:=.1f}% ({neg2}) are negative.".format(pos=len(valid_images_present)/len(valid_images)*100, pos2=len(valid_images_present),neg=len(valid_images_absent)/len(valid_images)*100, neg2=len(valid_images_absent)))
print("For the test stacks, {pos:=.1f}% ({pos2}) are positive and {neg:=.1f}% ({neg2}) are negative.".format(pos=len(test_images_present)/len(test_images)*100, pos2=len(test_images_present),neg=len(test_images_absent)/len(test_images)*100, neg2=len(test_images_absent)))

There are 9132 training stacks, 1142 validation stacks and 1142 test stacks.
For the training stacks, 50.2% (4583) are positive and 49.8% (4549) are negative.
For the validation stacks, 50.2% (573) are positive and 49.8% (569) are negative.
For the test stacks, 50.2% (573) are positive and 49.8% (569) are negative.


In [34]:
# Check for GPU support
use_cuda = torch.cuda.is_available()

In [35]:
# Check to see how many GPUs are available.
device = torch.device("cuda" if use_cuda else "cpu")
if use_cuda:
    num_devices = torch.cuda.device_count()
    print("{} GPUs found.".format(num_devices))
else:
    num_devices = 0

1 GPUs found.


# Plot sample image stacks

In [36]:
# Following https://github.com/mohakpatel/ImageSliceViewer3D, with modifications for plotting multiple stacks of color images. 
'''class ImageSliceViewer:
    """ 
    ImageSliceViewer is for viewing image stacks in jupyter or
    ipython notebooks. 
    
    User can interactively change the slice plane selection for the image.

    Argumentss:
    Volume = stack of color input images [num images, height, width, RGB]
    figsize = default(8,8), to set the size of the figure
    
    """
    
    def __init__(self, volume, figsize=(10,10)):
        self.volume = volume
        self.figsize = figsize
        self.v = [np.min(volume), np.max(volume)]
        
        # Call to select slice plane
        ipyw.interact(self.view_selection, view=ipyw.RadioButtons(
            options=['x-y'], value='x-y', 
            description='Slice plane selection:', disabled=False,
            style={'description_width': 'initial'}))
    
    def view_selection(self, view):
        # View the volume
        maxZ = self.volume.shape[0] - 1
        
        # Call to view a slice within the selected slice plane
        ipyw.interact(self.plot_slice, 
            z=ipyw.IntSlider(min=0, max=maxZ, step=1, continuous_update=False, 
            description='Image Slice:'))
        
    def plot_slice(self, z):
        # Plot slice for the given plane and slice
        fig=plt.figure(figsize=self.figsize)
        plt.imshow(self.volume[z,:,:,:], 
            vmin=self.v[0], vmax=self.v[1])
''';

In [37]:
'''def show_an_image_stack(dataset):
    fig = plt.figure(figsize = (10,10))
    axes = []
    # Grab a stack at random and build a volume.
    directories = os.listdir(dataset)
    directory = directories[np.random.randint(0,len(directories)-1)]
    files = os.listdir(os.path.join(dataset,directory))
    print("Plotting image stack {}".format(os.path.join(dataset,directory)))
    files.sort()
    imgArr = []
    for file in files:
        file = os.path.join(dataset,directory, file)
        imgArr.append(np.array(Image.open(file)))
    imgArr = np.array(imgArr)
    # Plot the volume within a subplot
    ImageSliceViewer(imgArr)
''';

In [38]:
# show_an_image_stack('Split_Data/train/Present')

# Imports for torch and DALI functions

In [39]:
# Define the model
class MM_DNN(nn.Module):
    def __init__(self, drop_prob=0.2, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.lr = lr
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc = nn.Linear(4*1024,256)
        self.fc2 = nn.Linear(256,64)
        self.fc3 = nn.Linear(64,2)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        x = x.view(-1)
        out = self.dropout(self.relu(self.fc(x)))
        out = self.dropout(self.relu(self.fc2(out)))
        out = self.fc3(out)
        # return the final output
        return out
    


In [40]:
# Use the PyTorch data loader here, way easier. 
def load_pickle(path):
    with open(path,'rb') as f:
        return pickle.load(f)
    
train_data = datasets.DatasetFolder(train_dir,load_pickle,'.pickle')
validation_data = datasets.DatasetFolder(valid_dir,load_pickle,'.pickle')
test_data = datasets.DatasetFolder(test_dir,load_pickle,'.pickle')
train_loader = torch.utils.data.DataLoader(train_data, batch_size = 1, shuffle=True, drop_last=True, num_workers=num_devices*4, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(validation_data, batch_size = 1, num_workers=num_devices*4, pin_memory=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 1, num_workers=num_devices*4, pin_memory=True)

In [41]:
saveName = 'Testing'

In [42]:
def train(net, train_loader, epochs=10, batch_size=2, lr=0.001, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: DNN network
        train_loader: PyTorch dataloader containing data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        lr: learning rate
        print_every: Number of steps for printing training and validation loss
    
    '''
    valid_batch = 1
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    if(use_cuda):
        net.cuda()
    all_valid_losses = [.6]
    counter = 0
    for e in range(epochs):
        
        for batch_idx, batch_data in enumerate(train_loader):
            print(batch_data)
            inputs = batch_data[0].view(12,batch_size,1024)
            inputs = inputs[0:4]
            targets = batch_data[1]
            if(use_cuda):
                inputs, targets = inputs.cuda(), targets.cuda()
            # zero accumulated gradients
            net.zero_grad()
            # get the output from the model
            output = net(inputs)
            # calculate the loss and perform backprop
            loss = criterion(output.view(batch_size,2), targets)
            loss.backward()
            opt.step()
            # loss stats
            if batch_idx % print_every == 0:
                # Get validation loss
                val_losses = []
                net.eval()
                correct = 0.
                total = 0.
                for val_batch_idx, batch_data in enumerate(valid_loader):                    
                    inputs, targets = batch_data[0].view(12,valid_batch,1024), batch_data[1]
                    inputs = inputs[0:4]
                    
                    if(use_cuda):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output = net(inputs)
                    val_loss = criterion(output.view(valid_batch,2), targets)
                    val_losses.append(val_loss.item())
                    pred = output.max(0,keepdim=True)[1]
                    # compare predictions to true label
                    correct += np.sum(np.squeeze(pred.eq(targets.data.view_as(pred))).cpu().numpy())
                    total += valid_batch
                
                net.train() # reset to train mode after iteration through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(batch_idx),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}...".format(np.mean(val_losses)),
                      "Val Accuracy: {:.1f}%".format(correct/total*100))
                
                if np.mean(val_losses) < np.min(all_valid_losses):
                    print("Validation loss dropped, saving model.")
                    torch.save(net,'DNNModel_' + saveName + '.pt')
                    all_valid_losses = np.append(all_valid_losses,np.mean(val_losses))

In [43]:
# define and print the net

net = MM_DNN()
#net = MM_LSTM(32,1)
print(net)

MM_DNN(
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=4096, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
  (relu): ReLU()
)


In [44]:
train(net,train_loader,50,1,print_every=6000,lr=0.0001)

[tensor([[[[1.8439e-04, 8.8264e-04, 5.8355e-03,  ..., 2.3206e-02,
           8.1967e-01, 1.7864e-01]],

         [[3.6308e-04, 2.2576e-03, 4.1814e-03,  ..., 3.5526e-02,
           1.0495e+00, 2.1366e-01]],

         [[3.5256e-04, 2.1856e-03, 4.1402e-03,  ..., 3.0623e-02,
           1.0869e+00, 1.8705e-01]],

         ...,

         [[3.5021e-04, 1.9837e-03, 4.0595e-03,  ..., 5.3062e-03,
           1.1212e+00, 2.8867e-01]],

         [[3.3980e-04, 1.8985e-03, 4.1095e-03,  ..., 1.2832e-02,
           1.0483e+00, 2.3769e-01]],

         [[3.3884e-04, 2.2236e-03, 4.0261e-03,  ..., 7.1390e-03,
           1.0691e+00, 2.4712e-01]]]]), tensor([0])]
Epoch: 1/50... Step: 0... Loss: 0.7046... Val Loss: 0.6966... Val Accuracy: 49.9%
[tensor([[[[3.1055e-04, 7.8797e-04, 5.8143e-03,  ..., 1.0325e-01,
           7.2517e-01, 0.0000e+00]],

         [[3.3028e-04, 7.5081e-04, 4.9344e-03,  ..., 1.7538e-01,
           5.8280e-01, 0.0000e+00]],

         [[3.3545e-04, 6.8349e-04, 4.7151e-03,  ..., 2.0211e-0

[tensor([[[[3.7602e-04, 2.5574e-04, 3.1689e-03,  ..., 2.3950e-01,
           8.9851e-03, 6.1423e-03]],

         [[1.9102e-04, 1.1027e-03, 1.5497e-03,  ..., 8.5099e-01,
           2.7757e-01, 0.0000e+00]],

         [[1.6799e-04, 8.6266e-04, 1.9356e-03,  ..., 2.1503e-01,
           9.0903e-02, 0.0000e+00]],

         ...,

         [[1.5472e-04, 9.2312e-04, 1.5564e-03,  ..., 2.0577e-01,
           4.8536e-02, 0.0000e+00]],

         [[1.5441e-04, 8.8822e-04, 1.6112e-03,  ..., 2.1116e-01,
           5.7313e-02, 0.0000e+00]],

         [[1.6291e-04, 6.8881e-04, 1.6451e-03,  ..., 1.7553e-01,
           5.9564e-02, 0.0000e+00]]]]), tensor([1])]
[tensor([[[[3.6273e-04, 1.9387e-03, 3.3049e-03,  ..., 6.0601e-02,
           1.0388e-01, 0.0000e+00]],

         [[3.3628e-04, 2.9955e-03, 2.5931e-03,  ..., 1.4176e-01,
           2.5706e-01, 0.0000e+00]],

         [[4.8392e-04, 2.4796e-03, 2.0160e-03,  ..., 4.1069e-01,
           4.3106e-01, 0.0000e+00]],

         ...,

         [[3.6116e-04, 1.0

[tensor([[[[2.9224e-04, 2.9575e-04, 3.8602e-03,  ..., 3.7889e-01,
           2.6745e-01, 7.5161e-04]],

         [[3.2797e-04, 5.4291e-04, 3.7552e-03,  ..., 3.0892e-01,
           2.7657e-01, 1.8012e-03]],

         [[3.3154e-04, 7.3142e-04, 3.4215e-03,  ..., 2.8588e-01,
           2.5392e-01, 1.2774e-02]],

         ...,

         [[2.7727e-04, 7.1774e-04, 3.0254e-03,  ..., 2.5542e-01,
           1.6267e-01, 0.0000e+00]],

         [[2.5812e-04, 2.6747e-04, 3.2704e-03,  ..., 6.1236e-01,
           6.7381e-02, 2.7136e-03]],

         [[3.1011e-04, 3.5618e-04, 3.5436e-03,  ..., 6.0033e-01,
           9.2755e-02, 1.6985e-02]]]]), tensor([1])]
[tensor([[[[2.8744e-04, 1.3596e-03, 6.2726e-03,  ..., 0.0000e+00,
           9.6857e-01, 7.5717e-03]],

         [[1.6165e-04, 4.1653e-04, 5.0330e-03,  ..., 0.0000e+00,
           3.1868e-01, 1.4236e-02]],

         [[2.2438e-04, 8.1920e-04, 3.9763e-03,  ..., 3.9044e-02,
           4.3955e-01, 4.6855e-03]],

         ...,

         [[2.8868e-04, 7.5

RuntimeError: shape '[12, 1, 1024]' is invalid for input of size 0

In [37]:
net = torch.load('DNNModel_' + saveName + '.pt')
stateDictName ='DNN_StateDict_' + saveName + '.pt'

In [38]:
torch.save(net.state_dict(), stateDictName)

In [39]:
net.load_state_dict(torch.load(stateDictName))

In [40]:
net.load_state_dict(torch.load('DNN_StateDict_S3_Iter_1.pt'))

# Zip up state dict, get over to S3

In [90]:
# The below lines are only needed if allowing SageMaker to perform inference on the model.
# I can walk you through that when you get there.

#tarName = 'DNNModel_' + saveName + '.tar.gz' 

In [91]:
#os.system('tar -cvzf ' + tarName + ' ' + stateDictName)
#os.system('aws s3 cp ' + tarName + ' s3://ypb-ml-images/' + tarName)

0